In [ ]:
import imaplib
import email
from email.header import decode_header

# Gmail IMAP settings
IMAP_SERVER = "imap.gmail.com"
IMAP_PORT = 993
EMAIL_ACCOUNT = "sampler@gmail.com"
EMAIL_PASSWORD = "zbgoaahzhvbtmwim"  # Use App Password, not Gmail password

def fetch_emails():
    """Fetch emails using IMAP."""
    try:
        # Connect to Gmail IMAP server
        mail = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
        mail.login(EMAIL_ACCOUNT, EMAIL_PASSWORD)

        # Select the mailbox (INBOX by default)
        mail.select("inbox")

        # Search for all emails
        status, messages = mail.search(None, "ALL")  # You can use 'UNSEEN' for unread emails

        # Convert the result into a list of email IDs
        email_ids = messages[0].split()

        print(f"Total Emails: {len(email_ids)}")

        # Fetch the latest 5 emails
        for email_id in email_ids[-10:]:
            status, msg_data = mail.fetch(email_id, "(RFC822)")

            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])

                    # Decode email subject
                    subject, encoding = decode_header(msg["Subject"])[0]
                    if isinstance(subject, bytes):
                        subject = subject.decode(encoding or "utf-8")

                    # Decode sender info
                    sender, encoding = decode_header(msg.get("From"))[0]
                    if isinstance(sender, bytes):
                        sender = sender.decode(encoding or "utf-8")

                    print(f"\nFrom: {sender}")
                    print(f"Subject: {subject}")

                    # Extract email body
                    if msg.is_multipart():
                        for part in msg.walk():
                            content_type = part.get_content_type()
                            content_disposition = str(part.get("Content-Disposition"))

                            # Extract the email body (only plain text, ignoring attachments)
                            if content_type == "text/plain" and "attachment" not in content_disposition:
                                body = part.get_payload(decode=True).decode()
                                print(f"Body:\n{body[:300]}...")  # Print first 300 chars
                    else:
                        body = msg.get_payload(decode=True).decode()
                        print(f"Body:\n{body[:300]}...")

        # Logout and close connection
        mail.logout()
    
    except Exception as e:
        print(f"Error: {e}")

# Run the function
fetch_emails()


In [ ]:
import imaplib
import email
import csv

# IMAP Configuration
IMAP_SERVER = "imap.gmail.com"
EMAIL_ID = "sampler@gmail.com"
PASSWORD = "zbgoaahzhvbtmwim"

# Connect to Gmail IMAP Server
mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL_ID, PASSWORD)
mail.select("inbox")

# Fetch latest 5 emails
status, messages = mail.search(None, "ALL")
email_ids = messages[0].split()[-5:]  # Get last 5 emails

emails = []  # Store email data

for email_id in email_ids:
    status, msg_data = mail.fetch(email_id, "(RFC822)")
    
    for response_part in msg_data:
        if isinstance(response_part, tuple):
            msg = email.message_from_bytes(response_part[1])
            sender = msg["From"]
            subject = msg["Subject"]

            # Extract email body
            if msg.is_multipart():
                body = ""
                for part in msg.walk():
                    if part.get_content_type() == "text/plain":
                        body = part.get_payload(decode=True).decode("utf-8", errors="ignore")
                        break
            else:
                body = msg.get_payload(decode=True).decode("utf-8", errors="ignore")

            emails.append({"sender": sender, "subject": subject, "body": body[:]})  # Limit body preview

# Save to CSV File
with open("emails.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Sender", "Subject", "Body"])  # Header row

    for email_data in emails:
        writer.writerow([email_data["sender"], email_data["subject"], email_data["body"]])

print("Emails saved to emails.csv")

# Close the connection
mail.logout()


Emails saved to emails.csv


('BYE', [b'LOGOUT Requested'])

In [ ]:
%pip install langgraph
%pip install langchain

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax

# Load pre-trained BERT model (fine-tuned on job classification)
MODEL_NAME = "bert-base-uncased"  

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME)
model.eval()

# Load email data
csv_file = "emails.csv"  # Update with actual CSV file
emails_df = pd.read_csv(csv_file)

# Function to classify emails
def classify_email(subject, body):
    text = f"{subject} {body}"
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    scores = softmax(outputs.logits, dim=1).numpy()[0]
    categories = ["ML", "UI", "QA"]  # Adjust based on your fine-tuned model
    return categories[scores.argmax()]

# Apply classification
emails_df["Category"] = emails_df.apply(lambda row: classify_email(row["Subject"], row["Body"]), axis=1)

# Save results
emails_df.to_csv("classified_emails.csv", index=False)
print("Classification completed and saved!")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Classification completed and saved!


Total Emails: 1195

From: "Amazon.in" <order-update@amazon.in>
Subject: Delivered: Your Amazon package has been delivered.
Category: Other
Body:
www.amazon.in
--------------------------------------------------------------------------------------------------------------------------------------------

Hi Syed,

Your package has been delivered!

Please rate your delivery experience.
https://www.amazon.in/deliveryfeedback?&ref_=tnp_email_del_fdbk&orderId=407-5432314-7901159&shipmentId=90907060591302


Return or replace items in Your Orders: 

https://www.amazon.in/gp/css/order-history?ie=UTF8&ref_=E_301

----------------------...

From: Unstop <noreply@unstop.news>
Subject: Syed Saleem, you have new job recommendation!
Category: Other
Body:
...

From: Unstop <noreply@unstop.news>
Subject: These mentors can help you get ahead in your career. Book a 1:1 call with them.
Category: Other
Body:
...

From: Ramya Gandikota <invitations@linkedin.com>
Subject: You have an invitation ✉️
Category: ML


In [21]:
import csv

# Job classification keywords
JOB_CATEGORIES = {
    "ML": ["machine learning", "deep learning", "AI", "artificial intelligence", "neural networks", "PyTorch", "TensorFlow"],
    "UI": ["UI/UX", "React", "Figma", "HTML", "CSS", "JavaScript", "Frontend"],
    "QA": ["testing", "Selenium", "automation", "test cases", "software testing", "QA engineer"]
}

def classify_email(body):
    """Classify the email into ML, UI, or QA based on keywords."""
    body_lower = body.lower()
    
    for category, keywords in JOB_CATEGORIES.items():
        if any(keyword in body_lower for keyword in keywords):
            return category
    
    return "Other"  # Default category if no match

def process_emails():
    """Read emails from CSV, classify them, and save the results."""
    classified_emails = []

    # Read emails from `emails.csv`
    with open("/Users/innovapathinc/Downloads/Agentic_AI/Agentic_AI/job_classification_project/data/emails.csv", "r", encoding="utf-8") as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row
        
        for row in reader:
            sender, subject, body = row[0], row[1], row[2]
            category = classify_email(body)
            classified_emails.append([sender, subject, body[:300], category])

    # Save classified emails to a new CSV
    with open("classified_emails.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Sender", "Subject", "Body (Shortened)", "Category"])
        writer.writerows(classified_emails)

    print("\nClassified emails saved to `classified_emails.csv` ✅")

# Run the function
process_emails()



Classified emails saved to `classified_emails.csv` ✅


In [28]:
import pandas as pd
data=pd.read_csv("/Users/innovapathinc/Downloads/Agentic_AI/emails.csv")
df=pd.DataFrame(data)
df.head(10)

,Sender,Subject,Body
0,Unstop <noreply@unstop.news>,=?utf-8?q?Syed_Saleem=2C_you_have_new_job_reco...,NaN
1,Unstop <noreply@unstop.news>,=?utf-8?q?These_mentors_can_help_you_get_ahead...,NaN
2,Ramya Gandikota <invitations@linkedin.com>,=?UTF-8?Q?You_have_an_invitation_=E2=9C=89=EF=...,Ramya is waiting for your response\r\n ...
3,LinkedIn <notifications-noreply@linkedin.com>,You have 1 new invitation,I hope this email finds you well.\r\n\r\nDear ...
4,MongoDB <mongodbteam@messages.mongodb.com>,Intro to Data Modeling on March 27th,"Hi Syed,\r\n\r\nA frequently asked question in..."
5,Team Unstop <noreply@dare2compete.news>,=?utf-8?q?=5BHiring=5D_500+_openings_at_Google...,NaN
6,"""Adobe Acrobat"" <mail@mail.adobe.com>",=?utf-8?B?VW5kZXJzdGFuZCBjb250cmFjdHMgQVNBUCB3...,\r\n\r\nView web version:\r\nhttps://t-info.ma...
7,Udemy <hello@students.udemy.com>,How do you like to learn? [Courses on sale now],Learn from your sofa. Learn from the bus. On m...
8,yamini n <invitations@linkedin.com>,=?UTF-8?Q?You_have_an_invitation_=E2=9C=89=EF=...,"yamini, Data Scientist from Lavu Tech Solution..."
9,Naukri Alerts <naukrialerts@naukri.com>,"=?UTF-8?Q?_=F0=9F=94=94_Syed,_check_out_jobs_a...",Job recommendations based on your Naukri.com p...
